![Bap the model! Make more data!](bap.gif)
# Bap the model, get more data!

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
import logging
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config, GPT2LMHeadModel, GPTNeoForCausalLM
import torch
import re

sys.path.append('..')
from model_manager import ModelManager

In [ ]:
# Tell pytorch to run this model on the GPU.
device_name = "cuda:0" if torch.cuda.is_available() else "cpu"
# device_name = "cpu"
device = torch.device(device_name)
logging.basicConfig(level=logging.INFO)

In [ ]:
model_folder = os.path.join("..", "models", "awsw_main")
model_manager = ModelManager(model_folder)

In [ ]:
end_token = "<|endoftext|>"
prompts = [
    ('PlayerReply c "Hey Kevin!" DragonReply Kv "Hey!"', "How are you?", "Kv"),
    ('PlayerReply c "Hey Adine!" DragonReply Ad "Hello."', "How are you?", "Ad"),
    ('PlayerReply c "Hey Maverick!" DragonReply Kv "Oh, it\'s you..."', "How are you?", "Mv"),
    ('PlayerReply c "Hey Remy, Anna told me about your love life." DragonReply Ry "My love life? It\'s non-existent."', "It isn\'t anymore...", "Ry"),
    ('PlayerReply c "Maverick..." DragonReply Mv "What is it..."', "I tried to sleep under your wing in my dreams but you strangled me and it turned into a fight nightmare.", "Mv"),
    ('PlayerReply c "You must be Anna!" DragonReply An "Hey, I heard about you"', "TheNoodle is cool, don't you think?", "An"),
]
fixed_replies = []
for (past, prompt, correct_dragon) in prompts:
    reply = model_manager.say(past, prompt)
    reply = reply.replace(end_token, "")
    current_name = reply[:reply.index(" ")]
    if current_name != correct_dragon:
        logging.warning(f"Reply got wrong name (should have been {correct_dragon} but is {current_name}). Repairing...")
        reply = reply[reply.index(" ") + 1:]
        reply = f"{correct_dragon} {reply}"
        fixed_replies.append(f'{past} PlayerReply c "{prompt}" DragonReply {reply}')
    print(f"Prompt: {prompt}\nReply: {reply}\n\n")
fixed_replies